In [2]:
# Let's start by retrieving some Magic card images using the Scryfall API
# https://scryfall.com/docs/api
# We'll be using the Scryfall python library
# https://github.com/NandaScott/Scrython

In [29]:
import requests
import json
import pprint
import time
import shutil
import os.path

URL='https://api.scryfall.com'
OUTDIR='/Users/agussman/Documents/DeepLens/deeplens-magiccards/data'


In [ ]:
r = requests.get('https://api.scryfall.com/cards/search?order=set&q=e:dom')

In [32]:
results = r.json()['data']
for result in results[:10]:
    #card = result
    #if 'all_parts' in result
    #print(result['image_uris']['png'])
    filename = "{}_{num:03d}.png".format(result['set'], num=int(result['collector_number']))
    print(filename)
    time.sleep(0.05)
    response = requests.get(r['png'], stream=True)
    with open(os.path.join(OUTDIR, r['set'], filename), 'wb') as out_file:
        shutil.copyfileobj(response.raw(), out_file)
    
    
    
#pprint.pprint(results[0:2])

dom_001.png


TypeError: 'Response' object is not subscriptable

In [4]:
import time

query = input("Type the name of the set: ")

time.sleep(0.05)
sets = scrython.sets.Sets()

for i in range(sets.data_length()):
    if sets.set_name(i) == query:
        print("Set code:", sets.set_code(i).upper())
        break
    else:
        continue

Type the name of the set: dom


RuntimeError: Cannot run the event loop while another loop is running